# Lab 5: Spark SQL Basics - Solutions

**Objective**: Master Spark SQL for querying structured data with familiar SQL syntax.

**Learning Outcomes**:
- Write complex SQL queries in Spark
- Understand Spark SQL execution engine
- Work with multiple data formats (CSV, Parquet, JSON)
- Create and manage temporary views
- Optimize SQL queries for performance

**Estimated Time**: 50 minutes

---

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pandas as pd
import time
import tempfile
import os
import shutil

spark = SparkSession.builder \
    .appName("Lab5-Spark-SQL-Solutions") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.logLevel", "ERROR") \
    .config("spark.sql.execution.arrow.maxRecordsPerBatch", "1000") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")  # Suppress warnings for cleaner output
# Also set log level for root logger to be extra sure
spark.sparkContext.setLogLevel("ERROR")

print(f"🚀 Spark SQL Lab Solutions - Version {spark.version}")

# Enhanced Spark UI URL display
ui_url = spark.sparkContext.uiWebUrl
print(f"Spark UI: {ui_url}")
print("💡 In GitHub Codespaces: Check the 'PORTS' tab below for forwarded port 4040 to access Spark UI")

## Part 1: SQL Fundamentals in Spark

In [ ]:
# Load multiple datasets
customers_df = spark.read.csv("../Datasets/customers.csv", header=True, inferSchema=True)
transactions_df = spark.read.csv("../Datasets/customer_transactions.csv", header=True, inferSchema=True)
products_df = spark.read.csv("../Datasets/product_catalog.csv", header=True, inferSchema=True)
iot_df = spark.read.parquet("../Datasets/iot_sensor_readings.parquet")

# Create temporary views
customers_df.createOrReplaceTempView("customers")
transactions_df.createOrReplaceTempView("transactions")
products_df.createOrReplaceTempView("products")
iot_df.createOrReplaceTempView("iot_sensors")

print("📊 Datasets registered as SQL views:")
print(f"  - customers: {customers_df.count():,} records")
print(f"  - transactions: {transactions_df.count():,} records")
print(f"  - products: {products_df.count():,} records")
print(f"  - iot_sensors: {iot_df.count():,} records")

**Exercise 1.1**: Write SQL queries for business analytics.

In [ ]:
# Solution: Business Intelligence Queries

# Query 1: Monthly sales trends
monthly_sales = spark.sql("""
    SELECT 
        DATE_FORMAT(transaction_date, 'yyyy-MM') as year_month,
        SUM(amount) as total_sales,
        COUNT(*) as transaction_count,
        AVG(amount) as avg_transaction_amount,
        COUNT(DISTINCT customer_id) as unique_customers
    FROM transactions
    GROUP BY DATE_FORMAT(transaction_date, 'yyyy-MM')
    ORDER BY year_month
""")

print("📈 Monthly Sales Trends:")
monthly_sales.show()

# Query 2: Customer lifetime value by state
customer_ltv = spark.sql("""
    SELECT 
        c.state,
        COUNT(DISTINCT c.customer_id) as customer_count,
        SUM(t.amount) as total_revenue,
        AVG(customer_total.total_spent) as avg_customer_ltv,
        PERCENTILE_APPROX(customer_total.total_spent, 0.5) as median_ltv,
        MAX(customer_total.total_spent) as max_ltv
    FROM customers c
    JOIN transactions t ON c.customer_id = t.customer_id
    JOIN (
        SELECT 
            customer_id,
            SUM(amount) as total_spent
        FROM transactions
        GROUP BY customer_id
    ) customer_total ON c.customer_id = customer_total.customer_id
    GROUP BY c.state
    ORDER BY total_revenue DESC
""")

print("\n💰 Customer LTV by State:")
customer_ltv.show()

# Query 3: Product performance analysis
product_performance = spark.sql("""
    WITH category_sales AS (
        SELECT 
            t.category,
            SUM(t.amount) as total_sales,
            COUNT(*) as transaction_count,
            AVG(t.amount) as avg_transaction,
            COUNT(DISTINCT t.customer_id) as unique_customers
        FROM transactions t
        GROUP BY t.category
    ),
    product_stats AS (
        SELECT 
            p.category,
            COUNT(*) as product_count,
            AVG(p.price) as avg_product_price,
            SUM(p.stock_quantity) as total_inventory
        FROM products p
        GROUP BY p.category
    )
    SELECT 
        cs.*,
        ps.product_count,
        ps.avg_product_price,
        ps.total_inventory,
        ROUND(cs.total_sales / cs.transaction_count, 2) as sales_per_transaction,
        ROUND(cs.total_sales / cs.unique_customers, 2) as sales_per_customer
    FROM category_sales cs
    LEFT JOIN product_stats ps ON cs.category = ps.category
    ORDER BY cs.total_sales DESC
""")

print("\n🛍️ Product Performance Analysis:")
product_performance.show()

# Validation
monthly_count = monthly_sales.count()
state_count = customer_ltv.count()
product_count = product_performance.count()

assert monthly_count > 0, "Should have monthly sales data"
assert state_count > 0, "Should have state-level LTV data"
assert product_count > 0, "Should have product performance data"

print(f"\n✓ Exercise 1.1 completed!")
print(f"📊 Generated {monthly_count} monthly reports, {state_count} state analyses, {product_count} category insights")

## Part 2: Advanced SQL Operations

In [ ]:
# Window functions in SQL
customer_rankings = spark.sql("""
    WITH customer_totals AS (
        SELECT 
            c.customer_id,
            c.name,
            c.state,
            SUM(t.amount) as total_spent,
            COUNT(t.transaction_id) as transaction_count
        FROM customers c
        JOIN transactions t ON c.customer_id = t.customer_id
        GROUP BY c.customer_id, c.name, c.state
    ),
    ranked_customers AS (
        SELECT 
            *,
            RANK() OVER (PARTITION BY state ORDER BY total_spent DESC) as state_rank,
            PERCENT_RANK() OVER (ORDER BY total_spent DESC) as percentile_rank
        FROM customer_totals
    )
    SELECT *
    FROM ranked_customers
    WHERE state_rank <= 3
    ORDER BY state, state_rank
""")

print("🏆 Top customers by state:")
customer_rankings.show(20)

**Exercise 2.1**: Complex analytical queries with CTEs and window functions.

In [ ]:
# Solution: Advanced Analytics Challenge

# Challenge 1: Customer Cohort Analysis
cohort_analysis = spark.sql("""
    WITH customer_cohorts AS (
        SELECT 
            customer_id,
            DATE_FORMAT(signup_date, 'yyyy-MM') as signup_cohort,
            signup_date
        FROM customers
    ),
    transaction_months AS (
        SELECT 
            t.customer_id,
            t.transaction_date,
            DATE_FORMAT(t.transaction_date, 'yyyy-MM') as transaction_month,
            cc.signup_cohort,
            cc.signup_date
        FROM transactions t
        JOIN customer_cohorts cc ON t.customer_id = cc.customer_id
    ),
    cohort_data AS (
        SELECT 
            signup_cohort,
            transaction_month,
            MONTHS_BETWEEN(TO_DATE(transaction_month, 'yyyy-MM'), TO_DATE(signup_cohort, 'yyyy-MM')) as months_since_signup,
            COUNT(DISTINCT customer_id) as active_customers,
            SUM(1) as transactions,
            SUM(amount) as revenue
        FROM (
            SELECT 
                tm.*,
                1 as amount  -- Simplified for demo
            FROM transaction_months tm
        ) tm_with_amount
        GROUP BY signup_cohort, transaction_month, MONTHS_BETWEEN(TO_DATE(transaction_month, 'yyyy-MM'), TO_DATE(signup_cohort, 'yyyy-MM'))
    ),
    cohort_sizes AS (
        SELECT 
            signup_cohort,
            COUNT(DISTINCT customer_id) as cohort_size
        FROM customer_cohorts
        GROUP BY signup_cohort
    )
    SELECT 
        cd.signup_cohort,
        cs.cohort_size,
        cd.months_since_signup,
        cd.active_customers,
        ROUND(cd.active_customers * 100.0 / cs.cohort_size, 2) as retention_rate
    FROM cohort_data cd
    JOIN cohort_sizes cs ON cd.signup_cohort = cs.signup_cohort
    WHERE cd.months_since_signup >= 0 AND cd.months_since_signup <= 12
    ORDER BY cd.signup_cohort, cd.months_since_signup
""")

print("📊 Customer Cohort Analysis:")
cohort_analysis.show(30)

# Challenge 2: IoT Sensor Analytics (using actual available columns)
sensor_insights = spark.sql("""
    WITH daily_sensor_stats AS (
        SELECT 
            DATE(timestamp) as reading_date,
            location,
            status,
            AVG(value) as avg_value,
            MIN(value) as min_value,
            MAX(value) as max_value,
            STDDEV(value) as value_stddev,
            COUNT(*) as reading_count,
            AVG(battery_level) as avg_battery,
            AVG(signal_strength) as avg_signal
        FROM iot_sensors
        GROUP BY DATE(timestamp), location, status
    ),
    sensor_stats_with_thresholds AS (
        SELECT 
            *,
            AVG(avg_value) OVER (PARTITION BY location, status) as overall_avg_value,
            STDDEV(avg_value) OVER (PARTITION BY location, status) as overall_value_stddev
        FROM daily_sensor_stats
    )
    SELECT 
        reading_date,
        location,
        status,
        ROUND(avg_value, 2) as avg_sensor_value,
        ROUND(min_value, 2) as min_sensor_value,
        ROUND(max_value, 2) as max_sensor_value,
        reading_count,
        ROUND(avg_battery, 2) as avg_battery_level,
        ROUND(avg_signal, 2) as avg_signal_strength,
        CASE 
            WHEN ABS(avg_value - overall_avg_value) > 2 * overall_value_stddev THEN 'ANOMALY'
            WHEN ABS(avg_value - overall_avg_value) > overall_value_stddev THEN 'WARNING'
            ELSE 'NORMAL'
        END as health_status
    FROM sensor_stats_with_thresholds
    ORDER BY reading_date DESC, location, status
""")

print("\n🔧 IoT Sensor Insights:")
sensor_insights.show(20)

# Challenge 3: Cross-dataset correlation (simplified to use available data)
sensor_location_correlation = spark.sql("""
    WITH daily_sensor_summary AS (
        SELECT 
            DATE(timestamp) as sensor_date,
            location,
            AVG(value) as avg_sensor_value,
            COUNT(*) as sensor_readings,
            CASE 
                WHEN AVG(value) > 75 THEN 'High'
                WHEN AVG(value) > 25 THEN 'Medium'
                ELSE 'Low'
            END as sensor_category
        FROM iot_sensors
        WHERE status = 'active'
        GROUP BY DATE(timestamp), location
    ),
    daily_sales AS (
        SELECT 
            DATE(transaction_date) as sales_date,
            SUM(amount) as daily_sales,
            COUNT(*) as transaction_count
        FROM transactions
        GROUP BY DATE(transaction_date)
    )
    SELECT 
        dss.sensor_category,
        COUNT(*) as days_count,
        AVG(ds.daily_sales) as avg_daily_sales,
        AVG(ds.transaction_count) as avg_transactions,
        AVG(dss.avg_sensor_value) as avg_sensor_reading
    FROM daily_sensor_summary dss
    JOIN daily_sales ds ON dss.sensor_date = ds.sales_date
    GROUP BY dss.sensor_category
    ORDER BY avg_daily_sales DESC
""")

print("\n📡 Sensor Activity vs Sales Correlation:")
sensor_location_correlation.show()

# Validation
cohort_count = cohort_analysis.count()
sensor_count = sensor_insights.count()
correlation_count = sensor_location_correlation.count()

assert cohort_count > 0, "Should have cohort analysis data"
assert sensor_count >= 0, "Should have sensor insights (may be 0 if no data)"
assert correlation_count >= 0, "Should have sensor-sales correlation (may be 0 if no matching data)"

print(f"\n✓ Exercise 2.1 completed!")
print(f"📈 Generated {cohort_count} cohort points, {sensor_count} sensor insights, {correlation_count} correlation patterns")

## Part 3: Performance Optimization and Data Formats

In [ ]:
# Compare query performance across different optimizations
print("⚡ Query Performance Analysis")

# Baseline query
baseline_query = """
    SELECT c.state, t.category, SUM(t.amount) as total_sales
    FROM customers c
    JOIN transactions t ON c.customer_id = t.customer_id
    GROUP BY c.state, t.category
    ORDER BY total_sales DESC
"""

# Optimized query with filtering
optimized_query = """
    SELECT c.state, t.category, SUM(t.amount) as total_sales
    FROM customers c
    JOIN transactions t ON c.customer_id = t.customer_id
    WHERE t.amount > 50 AND c.age BETWEEN 25 AND 65
    GROUP BY c.state, t.category
    HAVING SUM(t.amount) > 1000
    ORDER BY total_sales DESC
"""

# Time both queries
queries = [("Baseline", baseline_query), ("Optimized", optimized_query)]

for name, query in queries:
    start_time = time.time()
    result = spark.sql(query)
    count = result.count()
    exec_time = time.time() - start_time
    print(f"{name} query: {count} results in {exec_time:.4f}s")
    
    # Show execution plan
    print(f"{name} execution plan:")
    result.explain()
    print()

**Exercise 3.1**: Data format performance comparison and optimization.

In [ ]:
# Solution: Data Format Performance Test

# Create sample dataset for format comparison
sample_data = spark.sql("""
    SELECT c.*, t.amount, t.category, t.transaction_date
    FROM customers c
    JOIN transactions t ON c.customer_id = t.customer_id
    WHERE t.amount > 100
""")

temp_dir = tempfile.mkdtemp()
print(f"📁 Testing data formats in: {temp_dir}")

formats = [
    ("CSV", "csv", {"header": True}),
    ("Parquet", "parquet", {}),
    ("JSON", "json", {})
]

# Save in different formats and measure
format_metrics = {}

for format_name, format_type, options in formats:
    file_path = os.path.join(temp_dir, f"data.{format_type}")
    
    # Time the write operation
    start_time = time.time()
    if format_type == "csv":
        sample_data.coalesce(1).write.mode("overwrite").options(**options).csv(file_path)
    elif format_type == "parquet":
        sample_data.coalesce(1).write.mode("overwrite").parquet(file_path)
    elif format_type == "json":
        sample_data.coalesce(1).write.mode("overwrite").json(file_path)
    write_time = time.time() - start_time
    
    # Get directory size
    def get_dir_size(path):
        total = 0
        for dirpath, dirnames, filenames in os.walk(path):
            for filename in filenames:
                filepath = os.path.join(dirpath, filename)
                total += os.path.getsize(filepath)
        return total
    
    file_size = get_dir_size(file_path)
    
    # Time the read operation with proper schema handling
    start_time = time.time()
    if format_type == "csv":
        # For CSV, use inferSchema to get proper types
        read_df = spark.read.options(**options).option("inferSchema", "true").csv(file_path)
    elif format_type == "parquet":
        read_df = spark.read.parquet(file_path)
    elif format_type == "json":
        read_df = spark.read.json(file_path)
    
    # Force execution with count
    record_count = read_df.count()
    read_time = time.time() - start_time
    
    format_metrics[format_name] = {
        'write_time': write_time,
        'read_time': read_time,
        'file_size': file_size,
        'record_count': record_count
    }

# Display performance comparison
print("\n📊 Format Performance Comparison:")
for format_name, metrics in format_metrics.items():
    print(f"{format_name}:")
    print(f"  Write: {metrics['write_time']:.4f}s")
    print(f"  Read: {metrics['read_time']:.4f}s")
    print(f"  Size: {metrics['file_size']:,} bytes")
    print(f"  Records: {metrics['record_count']:,}")

# Query performance test across formats
print("\n⚡ Query Performance by Format:")

test_query = """
    SELECT category, 
           COUNT(*) as transaction_count,
           AVG(CAST(amount AS DOUBLE)) as avg_amount,
           SUM(CAST(amount AS DOUBLE)) as total_amount
    FROM format_test_table
    WHERE CAST(amount AS DOUBLE) > 150
    GROUP BY category
    ORDER BY total_amount DESC
"""

query_times = {}

for format_name, format_type, options in formats:
    file_path = os.path.join(temp_dir, f"data.{format_type}")
    
    # Read and register as temp table with proper schema handling
    if format_type == "csv":
        # For CSV, use inferSchema to get proper types
        df = spark.read.options(**options).option("inferSchema", "true").csv(file_path)
    elif format_type == "parquet":
        df = spark.read.parquet(file_path)
    elif format_type == "json":
        df = spark.read.json(file_path)
    
    df.createOrReplaceTempView("format_test_table")
    
    # Time the query
    start_time = time.time()
    query_result = spark.sql(test_query)
    result_count = query_result.count()
    query_time = time.time() - start_time
    
    query_times[format_name] = query_time
    print(f"{format_name} query: {result_count} results in {query_time:.4f}s")

# Find best performing format
best_read = min(format_metrics.items(), key=lambda x: x[1]['read_time'])
best_write = min(format_metrics.items(), key=lambda x: x[1]['write_time'])
smallest_size = min(format_metrics.items(), key=lambda x: x[1]['file_size'])
best_query = min(query_times.items(), key=lambda x: x[1])

print("\n🏆 Performance Summary:")
print(f"Fastest read: {best_read[0]} ({best_read[1]['read_time']:.4f}s)")
print(f"Fastest write: {best_write[0]} ({best_write[1]['write_time']:.4f}s)")
print(f"Smallest size: {smallest_size[0]} ({smallest_size[1]['file_size']:,} bytes)")
print(f"Fastest query: {best_query[0]} ({best_query[1]:.4f}s)")

# Cleanup
shutil.rmtree(temp_dir)

# Validation
assert len(format_metrics) == 3, "Should test 3 formats"
assert all(metrics['record_count'] > 0 for metrics in format_metrics.values()), "All formats should have records"
assert len(query_times) == 3, "Should have query times for all formats"

print("\n✓ Exercise 3.1 completed!")
print(f"🔬 Tested {len(formats)} data formats with comprehensive performance analysis")

## Summary: Spark SQL Mastery

### Key Capabilities Mastered:
1. **Standard SQL**: Full SQL-92 compliance with Spark extensions
2. **Advanced Functions**: Window functions, CTEs, complex joins, analytical functions
3. **Multiple Formats**: Optimal format selection based on use case
4. **Performance**: Catalyst optimizer leveraging and query optimization
5. **Integration**: Seamless DataFrame API and SQL integration

### Performance Insights:
| **Aspect** | **Best Practice** | **Performance Impact** |
|------------|-------------------|------------------------|
| **Data Format** | Parquet for analytics | 3-5x faster queries |
| **Filtering** | WHERE before GROUP BY | Reduces shuffle data |
| **Window Functions** | Partition appropriately | Minimizes data movement |
| **CTEs** | Organize complex queries | Better readability & optimization |
| **Joins** | Filter both sides first | Reduces join complexity |

### Best Practices Demonstrated:
- ✅ Use Parquet for analytical workloads (smallest size, fastest queries)
- ✅ Apply filters early in queries to reduce data volume
- ✅ Leverage window functions for advanced analytics
- ✅ Use CTEs for complex query organization and reusability
- ✅ Monitor query execution plans with EXPLAIN
- ✅ Test different approaches for performance optimization

In [ ]:
spark.stop()
print("🎉 Lab 5 completed! Spark SQL mastered.")
print("➡️  Next: Lab 6 - DataFrame Operations")